In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pyart
import os
import tempfile
from tint.data_utils import get_nexrad_keys, read_nexrad_key
from tint import Cell_tracks, animate
from tint.visualization import embed_mp4_as_gif
from boto.s3.connection import S3Connection
import gzip
from datetime import date, datetime, timedelta
from matplotlib import animation
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from glob import glob
from tint.tracks import Cell_tracks
from tint.visualization import animate


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# Function for pulling all keys between two dates at a chosen nexrad site.
def nexrad_site_datespan(start_date=None, start_date_time=None,
                         end_date=None, end_date_time=None, site=None):
    
    fmt = '%Y%m%d_%H%M%S'

    # Allows for the choice of now for the end date so current UTC is pulled.
    if end_date.upper() == 'NOW':
        e_d_selected = datetime.utcnow()
    else:
        e_d_selected = datetime.strptime(end_date + '_' + end_date_time, fmt)

    s_d = datetime.strptime(start_date + '_' + start_date_time, fmt)
    e_d_fixed = e_d_selected + timedelta(days=1)

    if s_d > e_d_selected:
            raise ValueError('You provided a start date' 
                             ' that comes after the end date.')

    times = []
    for timestamp in datespan((s_d), (e_d_fixed), delta=timedelta(days=1)):
        time = timestamp
        times += [datetime.strftime(time, '%Y/%m/%d/' + site.upper())]

    conn = S3Connection(anon=True)
    bucket = conn.get_bucket('noaa-nexrad-level2')

    # Get a list of files
    keys = []
    datetimes = []
    for time in times:
        bucket_list = list(bucket.list(time))
        for i in range(len(bucket_list)):
            this_str = str(bucket_list[i].key)
            if 'gz' in this_str:
                endme = this_str[-22:-3]
                fmt = '%Y%m%d_%H%M%S_V06'
                dt = datetime.strptime(endme, fmt)
                datetimes.append(dt)
                keys.append(bucket_list[i])

            if this_str[-3::] == 'V06':
                endme = this_str[-19::]
                fmt = '%Y%m%d_%H%M%S_V06'
                dt = datetime.strptime(endme, fmt)
                datetimes.append(dt)
                keys.append(bucket_list[i])

    # Code belows chooses all keys between the user's dates.

    d = {'keys': keys}
    key_object = pd.DataFrame(data=d, index=pd.to_datetime(datetimes))

    selected_keys = key_object.loc[s_d: e_d_selected, :]
    # radar_datetimes = selected_keys.index.tolist()
    data_keys = selected_keys['keys'].tolist()
    return data_keys

def datespan(start_date, end_date, delta=timedelta(days=1)):
    current_date = start_date
    while current_date < end_date:
        yield current_date
        current_date += delta


def get_grid(radar):
    """ Returns grid object from radar object. """
    grid = pyart.map.grid_from_radars(
        radar, grid_shape=(31, 500, 500),
        grid_limits=((0, 15000), (-200000, 200000), (-200000, 200000)),
        fields=['reflectivity'], gridding_algo='map_gates_to_grid',
        h_factor=0., nb=0.6, bsp=1., min_radius=200.)
    return grid

In [3]:
my_data_keys = nexrad_site_datespan(start_date='20180623',
                                         start_date_time='053000',
                                         end_date='20180623',
                                         end_date_time='180000',
                                         site='kvnx')

grids = []

print(len(my_data_keys))

a = 0

for key in my_data_keys:
    localfile = tempfile.NamedTemporaryFile()
    key.get_contents_to_filename(localfile.name)
    
    radar = pyart.io.read(localfile.name)
    
    grid = get_grid(radar)
    
    grids.append(grid)
    
    print(str(a) + ': ' + str(key)[41:-1])
    a = a+1

135
0: KVNX20180623_053207_V06
1: KVNX20180623_053757_V06
2: KVNX20180623_054346_V06
3: KVNX20180623_054937_V06
4: KVNX20180623_055527_V06
5: KVNX20180623_060118_V06
6: KVNX20180623_060708_V06
7: KVNX20180623_061258_V06
8: KVNX20180623_061848_V06
9: KVNX20180623_062437_V06
10: KVNX20180623_063029_V06
11: KVNX20180623_063618_V06
12: KVNX20180623_064205_V06
13: KVNX20180623_064752_V06
14: KVNX20180623_065341_V06
15: KVNX20180623_065932_V06
16: KVNX20180623_070522_V06
17: KVNX20180623_071113_V06
18: KVNX20180623_071702_V06
19: KVNX20180623_072251_V06
20: KVNX20180623_072842_V06
21: KVNX20180623_073431_V06
22: KVNX20180623_074021_V06
23: KVNX20180623_074613_V06
24: KVNX20180623_075203_V06
25: KVNX20180623_075753_V06
26: KVNX20180623_080342_V06
27: KVNX20180623_080933_V06
28: KVNX20180623_081523_V06
29: KVNX20180623_082112_V06
30: KVNX20180623_082703_V06
31: KVNX20180623_083254_V06
32: KVNX20180623_083845_V06
33: KVNX20180623_084436_V06
34: KVNX20180623_085026_V06
35: KVNX20180623_085616_V0

In [5]:
tracks_obj = Cell_tracks(field='reflectivity')

print(tracks_obj.params)
tracks_obj.get_tracks(iter(grids))

print(tracks_obj.tracks)

# Create animation in current working directory
animate(tracks_obj, grids, '/home/scarani/Desktop/test/tint_test_animation.mp4', alt=1500)

{'FIELD_THRESH': 32, 'MIN_SIZE': 8, 'SEARCH_MARGIN': 4000, 'FLOW_MARGIN': 10000, 'MAX_FLOW_MAG': 50, 'MAX_DISPARITY': 999, 'MAX_SHIFT_DISP': 15, 'ISO_THRESH': 8, 'ISO_SMOOTH': 3, 'GS_ALT': 1500}
shape (500, 500)
shape (105, 494)
shape (500, 484)
shape (76, 79)
shape (72, 478)
shape (62, 491)
shape (76, 500)
shape (62, 68)
shape (88, 447)
shape (68, 450)
Writing tracks for scan 0
shape (500, 500)
shape (500, 478)
shape (68, 458)
shape (68, 460)
shape (70, 106)
shape (84, 447)
shape (64, 449)
shape (62, 270)
shape (48, 210)
Writing tracks for scan 1
shape (500, 500)
shape (500, 474)
shape (74, 40)
shape (98, 460)
shape (70, 500)
shape (74, 44)
shape (66, 500)
shape (84, 92)
shape (72, 500)
shape (64, 500)
shape (88, 448)
shape (68, 449)
shape (41, 203)
Writing tracks for scan 2
shape (500, 500)
shape (500, 465)
shape (72, 92)
shape (64, 500)
shape (64, 431)
shape (84, 500)
shape (78, 500)
shape (62, 69)
shape (64, 45)
shape (64, 53)
shape (66, 500)
shape (90, 453)
shape (66, 370)
shape (

Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
Frame: 7
Frame: 8
Frame: 9
Frame: 10
Frame: 11
Frame: 12
Frame: 13
